# Complex Cash Flow

This notebook demonstrates how to access and use the functionalities of **Cash Flow** which are part of our **Yiedbook** module within LSEG Financial Analytics SDK. The notebook outlines the broadest complexity when performing cash flow, where all the possible set of arguments are present.

The main goal of this notebook is to explore and display an approach in fetching Cash Flow information by using any of 4 available methods:
1. Synchronous POST
2. Synchronous GET
3. Asynchronous POST
4. Asynchronous GET

## Imports

Import the following necessary modules:

- *post_cash_flow_sync* - main Synchronous POST method
- *CashFlowGlobalSettings* - Class wrapper for Global Settings 
- *CashFlowInput* - Class wrapper for Input parameters
- *json* - for display purposes 

In [1]:
from lseg_analytics.pricing.yield_book_rest import (
        post_cash_flow_sync,
        get_cash_flow_sync,
        post_cash_flow_async,
        get_cash_flow_async,
        get_result,
        CashFlowGlobalSettings, 
        CashFlowInput,
        Volatility,
        CurveTypeAndCurrency,
        LossSettings,
        RestPrepaySettings,
        CashflowFloaterSettings,
        CashflowMbsSettings,
        MuniSettings
)
import json as js

import time

## Data Preparation

Cash Flow Post methods consume two input arguments: 
- Global settings 
    - settings applicable in overall cash flow calculation
- Inputs
    - separate parameters to describe and enable specific CF information

In [2]:
# Select an ISIN or CUSIP ID of the instrument
identifier="31398GY86"

# Formulate Request body parameters - Global Settings
global_settings = CashFlowGlobalSettings(
            pricing_date="2025-01-13",
            use_previous_close=True,
            use_live_data=False,
            volatility=Volatility(
                type="Market",
                rate=1.11,
            ),
            retrieve_ppm_projection=False,
            core_logic_collateral="DEFAULT",
        )


# Formulate Request body parameters - Input 
input = CashFlowInput(
            identifier=identifier,
            id_type="CUSIP",
            curve=CurveTypeAndCurrency(
                curve_type="GVT",
                currency="USD",
                retrieve_curve=False,
                snapshot="EOD",
            ),
            settlement_type="MARKET",
            settlement_date="2025-01-15",
            custom_settlement="custom_settlement",
            par_amount="10000",
            # Loss Settings are not moddeled for all securities, so uncomment this section only for securities with Losses modeled
            #
            # loss_settings=LossSettings(
            #     default_type="SDA",
            #     default_rate=0.01,
            #     severity_type="MODEL",
            #     severity_rate=0.01,
            #     recovery_lag=1,
            #     delinquency_type="PASS",
            #     delinquency_rate=0.01,
            #     use_model_loan_modifications=False,
            #     ignore_insurance=False,
            # ),
            prepay=RestPrepaySettings(
                type="Model",
                rate=0.01,
            ),
            floater_settings=CashflowFloaterSettings(
                use_forward_index=True,
                forward_index_rate=0.01,
                calculate_to_maturity=False,
            ),
            muni_settings=MuniSettings(
                paydown_optional=False, 
                ignore_call_info=False, 
                use_stub_rate=False
            ),
            mbs_settings=CashflowMbsSettings(
                use_roll_info=False, 
                assume_call=False, 
                step_down_fail=False, 
                show_collateral_cash_flow=False
            )
)

## Synchronous POST Data retrieval

Once the input arguments are ready, they can be passed through the main method for data retrieval.

In [3]:
# Execute Post sync request with prepared inputs
cf_sync_post_response = post_cash_flow_sync(
                            global_settings=global_settings,
                            input=[input]
                        )

## Synchronous GET Data retrieval

As opposed to the POST method, the GET method requires the identifier itself and the par amount. The remaining arguments are optional.

In [4]:
# Formulate and execute the get request by using instrument ID and Par_amount
cf_sync_get_response = get_cash_flow_sync(
            id='999818LH',
            id_type="CUSIP",
            pricing_date="2021-01-01",
            par_amount="10000"
        )

## Asynchronous POST Data retrieval

The main difference between the Sync and Async approach is that Async methods allow execution of code on the user's side while waiting for a response. Once execution is complete, the user can 'collect' the results whenever they need them.

So this approach has 2 steps:
1. Provide input data to execution method
2. Use the request_id received in step 1 to retrieve the result.

In [5]:
# Request bond CF with async post
cf_async_post_response = post_cash_flow_async(
            global_settings=global_settings,
            input=[input]
        )

attempt = 1

while attempt < 10:

    from lseg_analytics.core.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_post_results_response = get_result(request_id_parameter=cf_async_post_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + cf_async_post_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{cf_async_post_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-12-03 12:43:52,949]	[MainThread]	[lseg_analytics.pricing.yield_book_rest]	[_functions.py:10384]	 Error get_result.
[ERROR]	[2025-12-03 12:43:52,950]	[MainThread]	[lseg_analytics.core]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-150621', 'status': 'RUNNING', 'responseType': 'CASHFLOW', 'message': 'The result is not ready yet for requestID:R-150621', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-150621


## Asynchronous GET Data retrieval

Difference between Sync and Async can be found in description above.

As Async execution is in queue-form it's not necessarily instantly finished. It can take time between submission of request and execution, and results are not in real-time.

Hence we present also the form of iterative results-request repetition to fetch the results as soon as they are ready.

**NOTE:** Alternative to this is wrapping the request in the job, and then using a method to check the job status. See the fundamentals notebook for information on jobs.

In [6]:
# Formulate and execute the get request by using instrument ID, Par_amount and job in which the calculation will be done
cf_async_get_response = get_cash_flow_async(
            id='999818LH',
            id_type="CUSIP",
            pricing_date="2021-01-01",
            par_amount="10000"
        )

# Due to async nature, code Will perform the fetch 10 times, as result is not always ready instantly, with 3 second lapse between attempts
attempt = 1

while attempt < 10:

    from lseg_analytics.core.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_get_results_response = get_result(request_id_parameter=cf_async_get_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + cf_async_get_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{cf_async_get_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-12-03 12:43:56,473]	[MainThread]	[lseg_analytics.pricing.yield_book_rest]	[_functions.py:10384]	 Error get_result.
[ERROR]	[2025-12-03 12:43:56,474]	[MainThread]	[lseg_analytics.core]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-150622', 'status': 'RUNNING', 'responseType': 'CASHFLOW', 'message': 'The result is not ready yet for requestID:R-150622', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-150622


## Display results

Results data is structured in JSON format so it can be easily displayed using print function.

### Synchronous POST results

In [7]:
# Print output to a file, as CF output is too long for terminal printout
print(js.dumps(cf_sync_post_response, indent=4), file=open('CF_output_Sync_Post.json', 'w+'))

### Synchronous GET results

In [8]:
# Print output to a file, as CF output is too long for terminal printout
print(js.dumps(cf_sync_get_response, indent=4), file=open('CF_output_Sync_Get.json', 'w+'))

### Asynchronous POST results

In [9]:
# Print results in json format
print(js.dumps(async_post_results_response, indent=4), file=open('CF_output_Async_Post.json', 'w+'))

### Asynchronous GET results

In [10]:
# Print results in json format
print(js.dumps(async_get_results_response, indent=4), file=open('CF_output_Async_Get.json', 'w+'))